In [2]:
import pickle
from os import path

In [7]:
DIR = "./output/206"
model_dir = path.join(DIR, "BRZ+TW+NASDAQ-Training-swap-nopunish-7d100-0-model.model")
model = pickle.load(open(model_dir, "rb"))

In [8]:
model

({'gamma': 0.99,
  'n_steps': 128,
  'vf_coef': 0.5,
  'ent_coef': 0.01,
  'max_grad_norm': 0.5,
  'learning_rate': <function stable_baselines.ppo2.ppo2.constfn.<locals>.func(_)>,
  'lam': 0.95,
  'nminibatches': 4,
  'noptepochs': 4,
  'cliprange': <function stable_baselines.ppo2.ppo2.constfn.<locals>.func(_)>,
  'cliprange_vf': 0.2,
  'verbose': 1,
  'policy': stable_baselines.common.policies.MlpPolicy,
  'observation_space': Box(6, 6),
  'action_space': Box(4,),
  'n_envs': 1,
  '_vectorize_action': False,
  'policy_kwargs': {}},
 OrderedDict([('model/pi_fc0/w:0',
               array([[-0.04589123,  0.08328784, -0.03131658, ..., -0.04549966,
                        0.19093336,  0.20499109],
                      [-0.00656073, -0.17641991,  0.15822113, ...,  0.4527115 ,
                       -0.24527296,  0.09952193],
                      [ 0.3218788 ,  0.16417654,  0.20253795, ..., -0.14080258,
                       -0.28167117,  0.1770732 ],
                      ...,
         